[![Open In Colab](https://colab.research.googleusercontent.com/assets/colab-badge.svg)](https://colab.research.google.com)

Si tu repo está en GitHub, usa este enlace editando USER/REPO/BRANCH:
[Open in Colab (GitHub)](https://colab.research.google.com/github/USER/REPO/blob/BRANCH/professor/pandas_v2/05_eda_groupby_multiindex.ipynb)



# 05 - EDA con pandas y GroupBy (incluye MultiIndex)

Objetivos:
- Cargar los datos limpios y explorar con `DataFrame.info`, `DataFrame.describe`.
- Usar `DataFrame.groupby` (una y varias claves) con agregaciones nombradas (`.agg`).
- Trabajar con `MultiIndex`: `reset_index`, `unstack`, `xs`, `swaplevel`, `sort_index`.
- Tabulados rápidos: `value_counts`, `crosstab`, `pivot_table` (con `margins`, `fill_value`).
- Agrupar por tiempo con `pd.Grouper` (si hay columna `fecha`).


In [ ]:
import pandas as pd
from pathlib import Path

path = Path("professor/pandas_v2/data/clean/dataset_limpio.csv")
print("Cargando:", path.as_posix())
df = pd.read_csv(path)
print("\nInfo rápida:")
print(df.info())
print("\nDescribe:")
print(df.describe(include="all", datetime_is_numeric=True))
print("\nPrimeras filas:")
print(df.head())


## GroupBy y agregaciones nombradas

Funciones clave en la celda de abajo:
- `DataFrame.groupby([...])` para agrupar por una o varias columnas.
- `.agg(...)` con agregaciones nombradas: `total=("monto","sum")`, etc.
- `as_index` controla si las claves se vuelven índice; `reset_index()` recupera columnas.
- Tips: `sort_values`, `sort_index`, `dropna`, `observed` (categorías).


In [ ]:
print("GroupBy por categoria_final:")
print(
    df.groupby("categoria_final").agg(
        total=("monto", "sum"),
        promedio=("monto", "mean"),
        conteo=("monto", "size"),
    )
)

print("\nGroupBy múltiple (categoria_final, nombre) y MultiIndex:")
res = df.groupby(["categoria_final", "nombre"]).agg(
    total=("monto", "sum"),
    trans=("monto", "size"),
)
print(res)

print("\n`unstack` para formato de reporte:")
print(res.unstack(0))


## GroupBy con `as_index=False` y más métricas

- `as_index=False` deja las claves como columnas.
- Agregaciones múltiples en diferentes columnas usando `.agg` con dict/tuplas.
- `sort_values` para ordenar el resultado de forma legible.


In [ ]:
print("GroupBy con as_index=False y múltiples agregaciones:")
res2 = (df.groupby("categoria_final", as_index=False)
          .agg(total=("monto","sum"), promedio=("monto","mean"), distintos_nombres=("nombre","nunique"))
          .sort_values("total", ascending=False))
print(res2)



## Operaciones útiles en MultiIndex

Funciones mostradas abajo:
- `xs(key, level=...)` para cortar una sección del índice por nivel.
- `swaplevel()` para reordenar niveles; `sort_index(level=...)` para ordenar.
- `unstack()` ya mostrado; `stack()` para el reverso.


In [ ]:
print("Extraemos con xs() el nivel categoria_final='A':")
try:
    print(res.xs('A', level=0))
except Exception as e:
    print("Nota: asegúrate de ejecutar la celda de groupby múltiple primero.")

print("\nReordenamos niveles con swaplevel() y ordenamos índice:")
try:
    swapped = res.swaplevel(0,1).sort_index(level=0)
    print(swapped.head())
except Exception as e:
    pass



## Frecuencias y tablas resumidas

- Frecuencias: `value_counts(normalize=True)`.
- `crosstab` para contingencias.
- `pivot_table` para resúmenes en formato wide (agregación incluida).


In [ ]:
print("Frecuencia de nombres (normalizada):")
print(df["nombre"].value_counts(normalize=True))

print("\nCrosstab categoria_final x nombre (conteos):")
print(pd.crosstab(df["categoria_final"], df["nombre"]))

print("\nPivot table de montos por categoria_final y nombre (suma):")
print(pd.pivot_table(df, index="categoria_final", columns="nombre", values="monto", aggfunc="sum"))
